In [9]:
import sys
sys.path.append("../../")

from yumipy import *
from prpy.planning import ompl, CBiRRTPlanner, chomp
from openravepy import *
import src.Robots as Robots
import Config

In [4]:
yumi_robot = YuMiRobot()
vel = YuMiRobot.get_v(50)
yumi_robot.left.set_speed(vel)
yumi_robot.left._motion_timeout = 1000
yumi_robot.left._comm_timeout = 1000
yumi_robot.left._process_timeout = 1000
vel = YuMiRobot.get_v(50)
yumi_robot.right.set_speed(vel)
yumi_robot.right._motion_timeout = 1000
yumi_robot.right._comm_timeout = 1000
yumi_robot.right._process_timeout = 1000

Process _YuMiEthernet-1:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
Process _YuMiEthernet-2:
  File "build/bdist.linux-x86_64/egg/yumipy/yumi_arm.py", line 67, in run
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self._reset_socket()
    self.run()
  File "build/bdist.linux-x86_64/egg/yumipy/yumi_arm.py", line 98, in _reset_socket
  File "build/bdist.linux-x86_64/egg/yumipy/yumi_arm.py", line 67, in run
    self._socket.connect((self._ip, self._port))
  File "/usr/lib/python2.7/socket.py", line 228, in meth
    self._reset_socket()
    return getattr(self._sock,name)(*args)
  File "build/bdist.linux-x86_64/egg/yumipy/yumi_arm.py", line 98, in _reset_socket
timeout: timed out
    self._socket.connect((self._ip, self._port))
  File "/usr/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
timeout

YuMiCommException: Request timed out: _REQ_PACKET(req='6 0 0 156 1 0 0 0 #', timeout=5, return_res=True)

In [ ]:
def execute_gripper_close(yumi_robot_hand,grip_force=10):
    return yumi_robot_hand.close_gripper(force=grip_force, width=0., no_wait=False, wait_for_res=True)

def execute_gripper_open(yumi_robot_hand):
    return  yumi_robot_hand.open_gripper(no_wait=False, wait_for_res=True)

def go_to_state(yumi_robot_hand,state):
    return  yumi_robot_hand.goto_state(state, wait_for_res=True)
    # print(res)

def execute_manip(yumi_robot_hand,trajectory):
    res = ''
    for i in trajectory.points:
        reordered_joints_in_radians = np.asarray(i.positions)[[0,1,3,4,5,6,2]] # TODO fix in openrave
        reordered_joints_in_degrees = np.degrees(reordered_joints_in_radians)
        state = YuMiState(list(reordered_joints_in_degrees))
        res = go_to_state(yumi_robot_hand,state)
    return res

In [ ]:
def planToConfiguration(robot, goal_config):
    '''
    find motion plan to given IK
    goal_config: IK to move arm to
    '''

    # planner = ompl.OMPLPlanner('RRTConnect')
    planner = CBiRRTPlanner(timelimit=30.0)
    simplifier = ompl.OMPLSimplifier()
    # Motion Planning to reach joint state value(s)
    # Get trajectory from planner based on type of goal config passed
    # ( config a.k.a ik solutions a.k.a joint states )
    try:
        # if len(goal_configs) == 1:
            # If goal is a single IK solution, then call PlanToConfiguration
        trajectory_object = planner.PlanToConfiguration(robot, goal_config, smoothingitrs=30)
        # else:
            # If goal is a list of IK solutions, then call PlanToConfigurations
        # trajectory_object = planner.PlanToConfigurations(robot, goal_configs)
        if hasattr(planner, 'default_ompl_args'):
            print("simplifying..")
            # If planner is from OMPL, then simplify the trajectory
            trajectory_object = simplifier.ShortcutPath(robot,trajectory_object)
    except Exception as e:#PlanningError:
        print("Exception ", e)
        # print(__file__.split('/')[-1],": Could not find motion plan")
        return None
    print("retiming..")
    # Retime and serialize the trajectory
    _ = planningutils.RetimeTrajectory(trajectory_object)
    trajectory_object = trajectory_object.serialize()
    return trajectory_object

In [10]:
env = Environment()
env.Load(Config.PROJ_DIR +"test_domains/Keva/keva_double_station_right.dae")
yumi_or_model = Robots.YumiRobot.YumiRobot(env)
yumi_or_robot = env.GetRobot("yumi")

In [ ]:
pose = ###
ik_sols = yumi_or_model.get_ik_solutions(pose,True)
go_to_state(yumi_robot.right,ik_sols[0])

In [ ]:
right_state = YuMiState([0.0, -130.0, 30.0, 0.0, 40., 0.0, -135.0])
ret = go_to_state(yumi_robot.right,right_state)

In [ ]:


pose_to_go = ## add pose here


ik_sols = yumi_or_model.get_ik_solutions(pose_to_go,True)
trajectory = planToConfiguration(yumi_or_robot,ik_sols[0])
execute_manip(yumi_robot.right,trajectory)

In [ ]:


pose_to_go = ## add pose here


ik_sols = yumi_or_model.get_ik_solutions(pose_to_go,True)
trajectory = planToConfiguration(yumi_or_robot,ik_sols[0])
execute_manip(yumi_robot.right,trajectory)

In [ ]:
pose = ###
ik_sols = yumi_or_model.get_ik_solutions(pose,True)
go_to_state(yumi_robot.right,ik_sols[0])

In [ ]:
right_state = YuMiState([0.0, -130.0, 30.0, 0.0, 40., 0.0, -135.0])
ret = go_to_state(yumi_robot.right,right_state)

In [ ]:


pose_to_go = ## add pose here


ik_sols = yumi_or_model.get_ik_solutions(pose_to_go,True)
trajectory = planToConfiguration(yumi_or_robot,ik_sols[0])
execute_manip(yumi_robot.right,trajectory)